In [ ]:
from yfinance import Ticker
def display_quote(symbol):
    quote = Ticker(symbol).history(period="1d")["Close"].values[0]
    return round(quote,2)

In [ ]:
display_quote("SPY")

C = call option price <br>
N = CDF of the normal distribution <br>
St= spot price of an asset <br>
K = strike price <br>
r = risk-free interest rate <br>
t = time to maturity <br>
σ = volatility of the asset <br>

In [ ]:
# Import modules
from scipy.stats import norm
from math import log, sqrt, exp
from datetime import datetime
from yfinance import Ticker
import quandl
    
def BlackScholes(symbol : str,
                 strike : float,
                 mat_date : str, #ex: 'Jun 1 2005  1:33PM'
                 call_put_flag : str = "c" or "p"):
 
    # Get data
    ticker = Ticker(symbol)
    price = ticker.history(period="1d")["Close"].values[0]
    vol = ticker.history(period="1Y")["Close"].std()
    div = ticker.info["dividendRate"]
    mat_time = ((datetime.strptime(mat_date, '%Y-%m-%d  %I:%M%p') - datetime.now()).days) / 365
    
    rf = quandl.get("FRED/DGS10")

    # Convert to fraction
    rf = rf / 100
    vol = vol / 100
    div = div / 100
    
    # Debugging
    # print(f"ticker : {ticker}, price : {price}, vol : {vol}, div : {div}, mat_time : {mat_time}")

    # Calculate d1 and d2
    d1 = (log(float(price) / strike)+((rf - div) + vol * vol / 2.) * mat_time) / ( vol * sqrt(mat_time))
    d2 = d1 - vol * sqrt(mat_time)
    
    # Calculate premium
    if call_put_flag =='c':
        return price * exp(-div * mat_time) * norm.cdf(d1) - strike * exp(-rf * mat_time) * norm.cdf(d2)
    
    elif call_put_flag == "p":
        return strike * exp(-rf * mat_time) * norm.cdf(-d2) -price * exp(-div * mat_time) * norm.cdf(-d1)
    
    else:
        raise CustomException("Error calculating option premium")

In [ ]:
from yfinance import Ticker

BlackScholes(call_put_flag = "c",
             symbol = "AAPL",
             strike = 148,
             mat_date = '2021-12-15 1:33PM',
             rf = 0.25)

In [ ]:
# import dependancies
from yfinance import Ticker
import datetime as dt
    
def OptionPrice(symbol : str,
                mat_date : str,
                strike : float,
                call_put_flag : str = "c" or "p"):
    
    ticker = Ticker(symbol)
    
    # get closest date
    option_dates = ticker.options
    option_dates = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in option_dates]
    date = min(option_dates, key=lambda x: abs(x - dt.datetime.strptime(mat_date, '%Y-%m-%d').date()))
    date = date.strftime('%Y-%m-%d')
    
    option_chain = None
    
    if call_put_flag == "c":
        option_chain = ticker.option_chain(date).calls
    
    elif call_put_flag == "p":
        option_chain = ticker.option_chain(date).puts
    
    else:
        raise CustomException("Error getting option premiums")
        
    
    closest_strike = min(option_chain["strike"], key=lambda x: abs(x - strike))
    
    option = option_chain[option_chain["strike"] == closest_strike]
    
    return option.to_json

In [ ]:
OptionPrice(symbol = "AAPL",
            mat_date = "2021-12-15",
            strike = 160,
            call_put_flag  = "c")

In [8]:
import quandl

quandl.get("FRED/DGS10").iloc[-1].Value

1.34